# 🧬 A ProtBERT Expedition with Dr. Aris

In this notebook, we once again follow **Dr. Aris**, a computational biologist who sees proteins not as mere chemical polymers, but as ancient biological manuscripts — sequences written in a molecular language refined by evolution.

In previous explorations, Dr. Aris turned to **ESM-2**, a powerful transformer trained to read this language at scale. With ESM-2, we successfully:

- Classified proteins as enzyme vs non-enzyme  
- Predicted Pfam families  
- Observed strong functional signal encoded in sequence embeddings  

The results were impressive.

But good science never stops at one model.

Today, Dr. Aris seeks a second opinion — another linguistic scholar trained to interpret protein grammar:

**ProtBERT**

ProtBERT approaches proteins as sentences composed of amino acid tokens, learning statistical structure from vast sequence corpora.

Our mission in this expedition:

- Use ProtBERT as a **frozen embedding generator**
- Train lightweight downstream classifiers (LR, SVM, MLP)
- Evaluate whether ProtBERT embeddings encode functional signal
- Compare fairly against our ESM-2 benchmarks

This is not about building the deepest neural network.

This is about understanding the representation.

Does ProtBERT truly grasp the grammar of biological function?

Let us find out.

### ⚙️ Environment Setup

In [1]:
# ================================
# Global Imports (Single Location)
# ================================

import os
import re
import math
import requests
import numpy as np
import pandas as pd
import torch
import time
from io import StringIO

from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    f1_score,
    accuracy_score
)

# ----------------
# Environment setup
# ----------------
os.environ["TOKENIZERS_PARALLELISM"] = "false"

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on:", device)

Running on: cuda


## 📂 Returning to the Library of Proteins

Dr. Aris does not redownload knowledge twice.

In our previous expedition, we carefully retrieved and stored a curated Swiss-Prot dataset from UniProt — a collection of proteins filtered for quality, completeness, and functional annotation.

Now, instead of querying the vast biological archive again, we simply reopen the manuscript we preserved.

In this step, Dr. Aris will:

- Load the stored UniProt dataset
- Remove incomplete or invalid sequences
- Enforce length constraints suitable for transformer models
- Optionally create a balanced benchmark subset
- Split the data into training and testing sets

The goal is simple:

Prepare a clean and reproducible foundation upon which ProtBERT can begin its linguistic analysis of proteins.

In [2]:
# ==========================================
# Load stored UniProt CSV -> clean -> balance -> split
# ==========================================

DATA_PATH = "uniprot_swissprot_enzyme_nonenzyme_FULL.csv"
df = pd.read_csv(DATA_PATH)
print("Loaded:", DATA_PATH, " shape:", df.shape)

# ---- Detect column names robustly ----
seq_col = "Sequence" if "Sequence" in df.columns else "sequence"
len_col = "Length"   if "Length"   in df.columns else "length"
acc_col = "Entry" if "Entry" in df.columns else ("Accession" if "Accession" in df.columns else "accession")

# --------------------------
# Cleaning
# --------------------------
df[seq_col] = df[seq_col].astype(str).str.strip()
df = df[df[seq_col].notna() & (df[seq_col].str.len() > 0)].copy()

df[len_col] = pd.to_numeric(df[len_col], errors="coerce")
df = df[df[len_col].between(50, 1024)].copy()

if acc_col in df.columns:
    before = len(df)
    df = df.drop_duplicates(subset=[acc_col]).copy()
    print(f"De-duplicated by {acc_col}: {before} -> {len(df)}")

# Keep only standard amino acids (+X allowed)
allowed = set("ACDEFGHIKLMNPQRSTVWYX")
df = df[df[seq_col].apply(lambda s: set(s).issubset(allowed))].copy()

df.reset_index(drop=True, inplace=True)

print("\nAfter cleaning:", df.shape)
print("Label counts:\n", df["label"].value_counts())

# --------------------------
# Optional fixed-size balanced benchmark
# --------------------------
TOTAL_N = 40000  # set to None to keep all data
if TOTAL_N is not None:
    per_class = TOTAL_N // 2

    n_pos = min(per_class, (df["label"] == 1).sum())
    n_neg = min(per_class, (df["label"] == 0).sum())

    df_pos = df[df["label"] == 1].sample(n=n_pos, random_state=SEED)
    df_neg = df[df["label"] == 0].sample(n=n_neg, random_state=SEED)

    df = pd.concat([df_pos, df_neg], ignore_index=True)\
           .sample(frac=1.0, random_state=SEED)\
           .reset_index(drop=True)

    print("\nAfter balancing:", df.shape)
    print("Label counts:\n", df["label"].value_counts())

# --------------------------
# Train/Test split
# --------------------------
train_df, test_df = train_test_split(
    df,
    test_size=0.20,
    random_state=SEED,
    stratify=df["label"]
)

print("\nTrain:", train_df.shape, "Test:", test_df.shape)

train_df.head()

/scratch/local/25278809/ipykernel_3304006/2353240010.py:6: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_PATH)


Loaded: uniprot_swissprot_enzyme_nonenzyme_FULL.csv  shape: (537346, 9)
De-duplicated by Entry: 537346 -> 537346

After cleaning: (536963, 9)
Label counts:
 label
0    270121
1    266842
Name: count, dtype: int64

After balancing: (40000, 9)
Label counts:
 label
0    20000
1    20000
Name: count, dtype: int64

Train: (32000, 9) Test: (8000, 9)


,Entry,Entry Name,Protein names,Organism,Length,EC number,Sequence,Reviewed,label
17005,P74737,RECA_SYNY3,Protein RecA (Recombinase A),Synechocystis sp. (strain ATCC 27184 / PCC 680...,354,NaN,MASTNISDREKALNAALAQIERSFGKGAIMRLGDATQMRVETISTG...,reviewed,0
19930,A1WXM7,SAHH_HALHL,Adenosylhomocysteinase (EC 3.13.2.1) (S-adenos...,Halorhodospira halophila (strain DSM 244 / SL1...,430,3.13.2.1,MSNQDYKVADISLADWGRKEIKIAESEMPGLMETRREFAAQKPLKG...,reviewed,1
8641,B7ZR65,SOX9A_XENLA,Transcription factor Sox-9-A,Xenopus laevis (African clawed frog),477,NaN,MNLLDPFMKMTEEQDKCMSGAPSPTMSDDSAGSPCPSGSGSDTENT...,reviewed,0
35026,P07435,OBP_BOVIN,Odorant-binding protein (OBP) (Olfactory mucos...,Bos taurus (Bovine),159,NaN,AQEEEAEQNLSELSGPWRTVYIGSTNPEKIQENGPFRTYFRELVFD...,reviewed,0
19704,P53690,MMP14_MOUSE,Matrix metalloproteinase-14 (MMP-14) (EC 3.4.2...,Mus musculus (Mouse),582,3.4.24.80,MSPAPRPSRSLLLPLLTLGTALASLGWAQGSNFSPEAWLQQYGYLP...,reviewed,1


## 🤖 Inviting ProtBERT to Read the Manuscripts

With the protein manuscripts cleaned, filtered, and carefully split into training and testing collections,  
Dr. Aris now turns to a new linguistic scholar of biology:

**ProtBERT**

Unlike traditional bioinformatics models, ProtBERT was trained to read proteins as if they were sentences —  
learning grammar, structure, and statistical patterns directly from vast sequence corpora.

But ProtBERT has its own reading conventions:

- Amino acids must be uppercase  
- Rare amino acids (`U`, `Z`, `O`, `B`) must be replaced with `X`  
- Sequences must be tokenized as space-separated letters  
  - Example: `"MKT..."` → `"M K T ..."`

Before ProtBERT can begin interpreting biological meaning,  
Dr. Aris must first prepare the sequences in the format ProtBERT understands.

### Load ProtBERT + Define Preprocessing

In [3]:
# ==========================================
# Load ProtBERT model + tokenizer
# ==========================================

MODEL_NAME = "Rostlab/prot_bert"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, do_lower_case=False)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)
model.eval()

print("Loaded:", MODEL_NAME)
print("Model running on:", device)


# ==========================================
# ProtBERT sequence preparation
# ==========================================

RARE_AA = re.compile(r"[UZOB]")

def protbert_prepare_sequence(seq: str) -> str:
    """
    Convert sequence to ProtBERT format:
    - Uppercase
    - Replace rare AAs with X
    - Space-separated tokens
    """
    seq = str(seq).strip().upper()
    seq = RARE_AA.sub("X", seq)
    return " ".join(list(seq))

Skipping import of cpp extensions due to incompatible torch version 2.9.1+cu128 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info


Loaded: Rostlab/prot_bert
Model running on: cuda


## 🧠 When ProtBERT Begins to Read

The manuscripts are prepared.  
The scholar is ready.

ProtBERT does not classify proteins directly.  
Instead, it transforms each sequence into a dense numerical representation —  
a **1024-dimensional embedding** that captures structural and functional patterns learned during pretraining.

In this step, Dr. Aris will:

- Tokenize each protein sequence
- Run a forward pass through ProtBERT
- Apply masked mean pooling over token embeddings
- Convert each protein into a fixed-length vector

These embeddings will serve as the foundation for downstream classifiers.

This is where language becomes representation.

### Generate ProtBERT Embeddings

In [5]:
# ==========================================
# Mean pooling helper
# ==========================================

@torch.no_grad()
def mean_pool(last_hidden_state: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
    mask = attention_mask.unsqueeze(-1).type_as(last_hidden_state)
    summed = (last_hidden_state * mask).sum(dim=1)
    counts = mask.sum(dim=1).clamp(min=1e-9)
    return summed / counts


# ==========================================
# Embedding extraction
# ==========================================

@torch.no_grad()
def build_protbert_embeddings(sequences, batch_size=8, max_length=512) -> np.ndarray:
    prepared = [protbert_prepare_sequence(s) for s in sequences]
    loader = DataLoader(prepared, batch_size=batch_size, shuffle=False)

    all_embs = []
    for batch in loader:
        toks = tokenizer(
            list(batch),
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_length
        ).to(device)

        out = model(**toks)
        pooled = mean_pool(out.last_hidden_state, toks["attention_mask"])
        all_embs.append(pooled.cpu().numpy())

    return np.vstack(all_embs)


# Detect sequence column
seq_col_train = "Sequence" if "Sequence" in train_df.columns else "sequence"

# Build embeddings
X_train = build_protbert_embeddings(train_df[seq_col_train].tolist(), batch_size=8)
X_test  = build_protbert_embeddings(test_df[seq_col_train].tolist(), batch_size=8)

y_train = train_df["label"].to_numpy().astype(int)
y_test  = test_df["label"].to_numpy().astype(int)

print("Train embeddings:", X_train.shape)
print("Test embeddings :", X_test.shape)
print("Labels shape     :", y_train.shape, y_test.shape)

Train embeddings: (32000, 1024)
Test embeddings : (8000, 1024)
Labels shape     : (32000,) (8000,)


## 🧪 Probing ProtBERT’s Knowledge

ProtBERT has now converted every protein into a fixed-length embedding.

But embeddings alone do not answer our scientific question.

Dr. Aris now performs a simple but powerful test:

If ProtBERT truly encodes functional meaning, then even **lightweight classifiers** should be able to separate:

- **Enzymes** (label = 1)
- **Non-enzymes** (label = 0)

So instead of training a huge deep network, we use small downstream models:

- Logistic Regression (linear probe)
- Linear SVM (margin-based probe)
- MLP (nonlinear probe)

This is not about building the most complex model.

This is about evaluating the representation.

In [6]:
def eval_binary_classifier(name, clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)

    # Probability-like scores for ROC/PR metrics
    if hasattr(clf, "predict_proba"):
        probs = clf.predict_proba(X_test)[:, 1]
    else:
        scores = clf.decision_function(X_test)
        probs = (scores - scores.min()) / (scores.max() - scores.min() + 1e-9)

    preds = (probs >= 0.5).astype(int)

    roc = roc_auc_score(y_test, probs)
    pr  = average_precision_score(y_test, probs)
    f1  = f1_score(y_test, preds)
    acc = accuracy_score(y_test, preds)

    print(f"{name:<14} ROC-AUC: {roc:.4f}")
    print(f"{name:<14} PR-AUC : {pr:.4f}")
    print(f"{name:<14} F1     : {f1:.4f}")
    print(f"{name:<14} Acc    : {acc:.4f}")
    print("-" * 60)

    return {"model": name, "roc_auc": roc, "pr_auc": pr, "f1": f1, "acc": acc}

### Train & Compare LR / SVM / MLP

In [7]:
results = []

# ProtBERT + Logistic Regression
clf_lr = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(max_iter=3000, n_jobs=-1))
])
results.append(eval_binary_classifier("ProtBERT+LR", clf_lr, X_train, y_train, X_test, y_test))

# ProtBERT + Linear SVM (calibrated)
base_svm = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", LinearSVC(max_iter=5000))
])
clf_svm = CalibratedClassifierCV(base_svm, method="sigmoid", cv=3)
results.append(eval_binary_classifier("ProtBERT+SVM", clf_svm, X_train, y_train, X_test, y_test))

# ProtBERT + MLP
clf_mlp = Pipeline([
    ("scaler", StandardScaler()),
    ("mlp", MLPClassifier(
        hidden_layer_sizes=(256, 128),
        activation="relu",
        alpha=1e-4,
        max_iter=200,
        early_stopping=True,
        n_iter_no_change=10,
        random_state=SEED
    ))
])
results.append(eval_binary_classifier("ProtBERT+MLP", clf_mlp, X_train, y_train, X_test, y_test))

pd.DataFrame(results)

ProtBERT+LR    ROC-AUC: 0.9729
ProtBERT+LR    PR-AUC : 0.9709
ProtBERT+LR    F1     : 0.9188
ProtBERT+LR    Acc    : 0.9191
------------------------------------------------------------


/apps/jupyter/6.5.4/lib/python3.10/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/apps/jupyter/6.5.4/lib/python3.10/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/apps/jupyter/6.5.4/lib/python3.10/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/apps/jupyter/6.5.4/lib/python3.10/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/apps/jupyter/6.5.4/lib/python3.10/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5

ProtBERT+SVM   ROC-AUC: 0.9730
ProtBERT+SVM   PR-AUC : 0.9711
ProtBERT+SVM   F1     : 0.9182
ProtBERT+SVM   Acc    : 0.9186
------------------------------------------------------------
ProtBERT+MLP   ROC-AUC: 0.9895
ProtBERT+MLP   PR-AUC : 0.9885
ProtBERT+MLP   F1     : 0.9551
ProtBERT+MLP   Acc    : 0.9555
------------------------------------------------------------


,model,roc_auc,pr_auc,f1,acc
0,ProtBERT+LR,0.972919,0.970926,0.918770,0.919125
1,ProtBERT+SVM,0.972957,0.971094,0.918247,0.918625
2,ProtBERT+MLP,0.989535,0.988521,0.955051,0.955500


# 🧭 Reflections from the Expedition

ProtBERT has spoken.

From raw amino acid sequences — nothing more than strings of letters — it constructed dense representations that allowed simple classifiers to distinguish enzymes from non-enzymes with strong performance.

This is the remarkable part.

We did not fine-tune ProtBERT.
We did not modify its internal weights.
We merely asked it to *encode*.

And yet, the embeddings carried functional signal.

For Dr. Aris, this confirms an important insight:

Protein language models are not memorizing sequences —  
they are learning structure in biological grammar.

How does ProtBERT compare to ESM-2?

- If performance is similar, it suggests convergent representation learning across architectures.
- If ESM-2 performs better, it may reflect architectural scaling or training corpus differences.
- If ProtBERT competes closely, it reinforces the robustness of transformer-based protein embeddings.

But the deeper takeaway is not which model wins.

It is this:

Functional meaning is encoded in sequence alone.

Transformers, when trained at scale, can uncover that meaning.

And now, Dr. Aris has two scholars in his library 
ESM-2 and ProtBERT —  
ready for future expeditions into biological language.